# The Factory Method Pattern and Its Implementation in Python

https://realpython.com/factory-method-python/

Factory Method is a creational design pattern used to create concrete implementations of a common interface.

Seperates *creation* of object from code that depends on the interface of the object

It can replace complex and ugly `if/elif/else` statements

In [1]:
# song serializer demo - no factory
import json
import xml.etree.ElementTree as et

class Song:
    def __init__(self, song_id, title, artist):
        self.song_id = song_id
        self.title = title
        self.artist = artist
        
class SongSerializer:
    def serialize(self, song, format):
        if format == 'JSON':
            song_info = {
                'id':song.song_id,
                'title':song.title,
                'artist':song.artist
            }
            return json.dumps(song_info)
        elif format == 'XML':
            song_info = et.Element('song', attrib={'id':song.song_id})
            title = et.SubElement(song_info, 'title')
            title.text = song.title
            artist = et.SubElement(song_info, 'artist')
            artist.text = song.artist
            return et.tostring(song_info, encoding='unicode')
        else:
            raise ValueError(format)

In [3]:
song = Song('1', 'Water of Love', 'Dire Straits')
serializer = SongSerializer()
serializer.serialize(song, 'JSON')

'{"id": "1", "title": "Water of Love", "artist": "Dire Straits"}'

In [4]:
serializer.serialize(song, 'XML')

'<song id="1"><title>Water of Love</title><artist>Dire Straits</artist></song>'

the code above is doing too much. The single responsibility principle states that a module, a class, or even a method should have a single, well-defined responsibility. It should do just one thing and have only one reason to change.

the serialize method here is:
* interpreting the format
* directing the correct path based on the format
* creating the output

because of this the `serialize` method will need to change whenever
* a new format is introduced
* the `Song` changes
* when a format changes 

## common interface
The code above converts a song object to its string representation using a different format in each logical path. 
1. Look for a common interface that can be used to replace each of the paths, 
2. provide separate implementations for each logical path
3. provide a separate component that decides how to implement by evaulating `format`, and returns the implementation

In [9]:
class SongSerializer:
    def serialize(self, song, format):
        if format == 'JSON':
            return self._serialize_to_JSON(song)
        elif format == 'XML':
            return self._serialize_to_XML(song)
        else:
            raise ValueError(format)
            
    def _serialize_to_JSON(self, song):        
            song_info = {
                'id':song.song_id,
                'title':song.title,
                'artist':song.artist
            }
            return json.dumps(song_info)
        
    def _serialize_to_XML(self, song):
            song_info = et.Element('song', attrib={'id':song.song_id})
            title = et.SubElement(song_info, 'title')
            title.text = song.title
            artist = et.SubElement(song_info, 'artist')
            artist.text = song.artist
            return et.tostring(song_info, encoding='unicode')

In [10]:
song = Song('1', 'Water of Love', 'Dire Straits')
serializer = SongSerializer()
serializer.serialize(song, 'JSON')

'{"id": "1", "title": "Water of Love", "artist": "Dire Straits"}'

In [11]:
serializer.serialize(song, 'XML')

'<song id="1"><title>Water of Love</title><artist>Dire Straits</artist></song>'

## Factory

provide a separate component with the responsibility to decide which concrete implementation should be used based on some specified parameter.

add a new method `._get_serializer()` that takes the desired `format` - it returns a *function*, not data

In [12]:
class SongSerializer:
    def serialize(self, song, format):
        serializer = self._get_serializer(format)
        return serializer(song)
            
    def _get_serializer(self, format):
        if format == 'JSON':
            return self._serialize_to_JSON
        elif format == 'XML':
            return self._serialize_to_XML
        else:
            raise ValueError(format)
            
    def _serialize_to_JSON(self, song):        
            song_info = {
                'id':song.song_id,
                'title':song.title,
                'artist':song.artist
            }
            return json.dumps(song_info)
        
    def _serialize_to_XML(self, song):
            song_info = et.Element('song', attrib={'id':song.song_id})
            title = et.SubElement(song_info, 'title')
            title.text = song.title
            artist = et.SubElement(song_info, 'artist')
            artist.text = song.artist
            return et.tostring(song_info, encoding='unicode')

In [13]:
song = Song('1', 'Water of Love', 'Dire Straits')
serializer = SongSerializer()
serializer.serialize(song, 'JSON')

'{"id": "1", "title": "Water of Love", "artist": "Dire Straits"}'

In [14]:
serializer.serialize(song, 'XML')

'<song id="1"><title>Water of Love</title><artist>Dire Straits</artist></song>'

## Compenents and additional pattern

Here:
* `serialize` is the *client* - it depends on an interface to complete its task
* the two funcs that take the song and return a string rep are *product* components, the implementation.
* `_get_serializer` is the *creator* component - it decides what implementation should be used.

Note that the creators and product don't use the self parameter. As rule of thumb, this is a sign you should take the methods out of the class. 

In [20]:
class SongSerializer:
    def serialize(self, song, format):
        serializer = get_serializer(format)
        return serializer(song)
            
def get_serializer(format):
    if format == 'JSON':
        return _serialize_to_JSON
    elif format == 'XML':
        return _serialize_to_XML
    else:
        raise ValueError(format)
            
def _serialize_to_JSON(song):        
    song_info = {
        'id':song.song_id,
        'title':song.title,
        'artist':song.artist
    }
    return json.dumps(song_info)
        
def _serialize_to_XML(song):
    song_info = et.Element('song', attrib={'id':song.song_id})
    title = et.SubElement(song_info, 'title')
    title.text = song.title
    artist = et.SubElement(song_info, 'artist')
    artist.text = song.artist
    return et.tostring(song_info, encoding='unicode')

In [21]:
song = Song('1', 'Water of Love', 'Dire Straits')
serializer = SongSerializer()
serializer.serialize(song, 'JSON')

'{"id": "1", "title": "Water of Love", "artist": "Dire Straits"}'